### 4. Working with Key/Value Pairs

We will look at a special type of RDD called Key Value Pair RDD which exposes some new set of operations and is most commonly used type of RDD when we need to perform some analytics and aggegation on large set of data. We will see how to partition the data based on key to ensure commonly accesses data resides together on the same partition and there are no shuffles involved in moving large amount of data across the machines over the network. 

These types of RDDs have a special name called ``PairRDD`` and has special features that let us perform aggregation by the key and also join two RDD based on the key. Let us read the a text file and create a ``PairRDD`` using the lower cased first word of each line in the text file and then perform some aggregation operations on them

In [36]:
val src = scala.io.Source.fromURL("https://raw.githubusercontent.com/apache/spark/master/README.md")
val contentRDD = sc.parallelize(src.mkString.split("\n"))
val pairRDD = contentRDD.map(x => (x.split(" ")(0).toLowerCase, 1))
println("Top 10 Words aggregated with their descending value of counts are " + 
pairRDD.reduceByKey((x, y) => x + y).takeOrdered(10)(Ordering.by(x => -x._2)).toList)


Top 10 Words aggregated with their descending value of counts are List((,47), (##,9), (spark,4), (please,4), (you,3), (for,3), (and,2), (to,2), (can,2), (hadoop,,1))



The ``reduceByKey`` is similar to ``reduce`` action we saw previously, except that the reduce is called on values of same key element and **``reduceByKey`` is a transformation**. Note that similar to ``reduce`` the return type of the ``reduceByKey`` is same as the type of the ***value*** in the key value pair. No where in the ``reduceByKey`` we know whats the key associated with the values being reduced. Spark automatically gives an RDD with the key and the reduced values of the key. This special type of reduce function is available only in key value type of RDD.

Since we resulting RDD is a tuple and while we take the top 10 values we want to order by the value, we used ``Ordering.by(x => -x._2)`` for ordering to sort numbers by the second element (value) of the tuple rather than the key (the first word of the line). Since ``Ordering.by`` will sort the numbers by ascending order and we wish to sort the numbers by descending value of the count, we simply flip the sign of the value.

Let us create a small RDD and studying some simple transformation we get in ``PairRDD``

In [41]:
val pairSample = sc.parallelize(List((1, 2), (3, 4), (3, 6)))
println("1. Reducing by key for adding the values for similar key: " + pairSample.reduceByKey(_ + _).collect.toList)
println("2. Group by key:  " + pairSample.groupByKey().collect.toList)
println("3. mapValue by key:  " + pairSample.mapValues(_ + 10).collect.toList)
println("4. flatMapValue by key:  " + pairSample.flatMapValues(x => x to (x + 2)).collect.toList)
println("5. Keys of pair RDD:  " + pairSample.keys.collect.toList)
println("6. Values of pair RDD:  " + pairSample.values.collect.toList)
println("7. sortByKey of pair RDD:  " + pairSample.sortByKey().collect.toList)

1. Reducing by key for adding the values for similar key: List((1,2), (3,10))
2. Group by key:  List((1,CompactBuffer(2)), (3,CompactBuffer(4, 6)))
3. mapValue by key:  List((1,12), (3,14), (3,16))
4. flatMapValue by key:  List((1,2), (1,3), (1,4), (3,4), (3,5), (3,6), (3,6), (3,7), (3,8))
5. Keys of pair RDD:  List(1, 3, 3)
6. Values of pair RDD:  List(2, 4, 6)
7. sortByKey of pair RDD:  List((1,2), (3,4), (3,6))



- **reduceByKey**: We have already seen this, after reduceByKey, the RDD is guaranteed not to have tuples with duplicate keys. The values are reduced by the function passed to ``reduceByKeys`` in this case additition.
- **groupByKey**: Similar to ``reduceByKey`` after invoking this method, the tuples in the RDD are guaranteed not to have duplicate keys. The value assiciated with the key is a buffer (a sequence of values) holding all the values assiciated with the key. Thus ``RDD[K, V].groupByKey() -> RDD[K, Seq[V]]``
- **mapValue**: applies the function to the values of in the key value pair essentially tranforming ``(k, v) -> (k, f(v))``
- **flatMapValue**: applies a function ``f(V) => Traversable[U]`` on the values of pair ``RDD[(K, V)]`` to emit the pair ``(K, U)`` for each value in ``Traversable[U]`` 
- **keys**: Simply returns the RDD of keys. ``RDD[(K, V)] => RDD[K]``
- **values**: Simply returns the RDD of values. ``RDD[(K, V)] => RDD[V]``
- **sortByKeys**: Creates an RDD which sorts the RDD by the keys.

---

We will now look at some operations on pair RDDs.

In [49]:
val anotherPairRDD = sc.parallelize(List((3, 9), (4, 10)))
println("1. Join two RDDs" + pairSample.join(anotherPairRDD).collect.toList)
println("2. Left join two RDDs" + pairSample.leftOuterJoin(anotherPairRDD).collect.toList)
println("3. Left join two RDDs" + pairSample.rightOuterJoin(anotherPairRDD).collect.toList)
println("4. Cogroup join two RDDs" + pairSample.cogroup(anotherPairRDD).collect.toList)

1. Join two RDDsList((3,(4,9)), (3,(6,9)))
2. Left join two RDDsList((1,(2,None)), (3,(4,Some(9))), (3,(6,Some(9))))
3. Left join two RDDsList((4,(None,10)), (3,(Some(4),9)), (3,(Some(6),9)))
4. Cogroup join two RDDsList((4,(CompactBuffer(),CompactBuffer(10))), (1,(CompactBuffer(2),CompactBuffer())), (3,(CompactBuffer(4, 6),CompactBuffer(9))))



- **join**: This operation reduces the result to only those elements whose keys are present in both the RDDs. Thus ``RDD[K, V1] join RDD [K, V2] -> RDD[K, (V1, V2)]``. If we have multiple values of K in either RDDs, we will get all possible combinations of V1 and V2 for the same keys.
- **leftOuterJoin**: Retains all keys in the left RDD. Thus ``RDD[K, V1] leftOuterJoin RDD [K, V2] -> RDD[K, (V1, V2)] -> RDD[K, (V1, Option[V2])]``. The ``Option`` will be ``Some[V2]`` if the matching key is present in right RDD else ``None``.
- **rightOuterJoin**: Similar to leftOuterJoin, except that the value tuple is ``(Option[V1], V2)`` instead of ``(V1, Option[V2])``
- **cogroup**: This is similar to ``groupByKey`` operation on a single RDD except that the result RDD is  ``RDD[K, (Seq[V1], Seq[V2])]``. If the key is not present in either RDD, the corresponding Seq will be empty.